In [1]:
from SmartApi.smartConnect import SmartConnect
import pyotp, time
from datetime import datetime, timedelta

args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
}

smartApi = SmartConnect(args["api_key"])
try:
    totp = pyotp.TOTP(args["token"]).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e
data = smartApi.generateSession(args["username"], args["pwd"], totp)
print(data)
if data['status'] == False:
    logger.error(data)
    raise Exception("Authentication failed")
else:
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    feedToken = smartApi.getfeedToken()
    profile = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)


[I 251201 17:40:30 smartConnect:124] in pool


{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'AAAF620196', 'name': 'A JAIADITHYA', 'email': '', 'mobileno': '', 'exchanges': ['nse_fo', 'nse_cm', 'cde_fo', 'ncx_fo', 'bse_fo', 'bse_cm', 'mcx_fo'], 'products': ['MARGIN', 'MIS', 'NRML', 'CNC', 'CO', 'BO'], 'lastlogintime': '', 'broker': '', 'jwtToken': 'Bearer eyJhbGciOiJIUzUxMiJ9.eyJ1c2VybmFtZSI6IkFBQUY2MjAxOTYiLCJyb2xlcyI6MCwidXNlcnR5cGUiOiJVU0VSIiwidG9rZW4iOiJleUpoYkdjaU9pSlNVekkxTmlJc0luUjVjQ0k2SWtwWFZDSjkuZXlKMWMyVnlYM1I1Y0dVaU9pSmpiR2xsYm5RaUxDSjBiMnRsYmw5MGVYQmxJam9pZEhKaFpHVmZZV05qWlhOelgzUnZhMlZ1SWl3aVoyMWZhV1FpT2pnc0luTnZkWEpqWlNJNklqTWlMQ0prWlhacFkyVmZhV1FpT2lJeU4yUXhNemt3WlMwME56STFMVE0wTURFdFlUSmpNaTFpWW1Zd016UXhOV0ZqTm1VaUxDSnJhV1FpT2lKMGNtRmtaVjlyWlhsZmRqSWlMQ0p2Ylc1bGJXRnVZV2RsY21sa0lqbzRMQ0p3Y205a2RXTjBjeUk2ZXlKa1pXMWhkQ0k2ZXlKemRHRjBkWE1pT2lKaFkzUnBkbVVpZlN3aWJXWWlPbnNpYzNSaGRIVnpJam9pWVdOMGFYWmxJbjE5TENKcGMzTWlPaUowY21Ga1pWOXNiMmRwYmw5elpYSjJhV05sSWl3aWMzVmlJam9pUVVGQlJqWXlNREU1TmlJc0l

In [2]:
import json

pcr_data = smartApi.putCallRatio()
# Pretty print the PCR data
print(json.dumps(pcr_data, indent=4))

{
    "status": true,
    "message": "SUCCESS",
    "errorcode": "",
    "data": [
        {
            "pcr": 0.93,
            "tradingSymbol": "FINNIFTY30DEC25FUT"
        },
        {
            "pcr": 0.63,
            "tradingSymbol": "ICICIBANK30DEC25FUT"
        },
        {
            "pcr": 0.69,
            "tradingSymbol": "BAJAJFINSV30DEC25FUT"
        },
        {
            "pcr": 0.69,
            "tradingSymbol": "RELIANCE30DEC25FUT"
        },
        {
            "pcr": 0.66,
            "tradingSymbol": "INFY30DEC25FUT"
        },
        {
            "pcr": 0.54,
            "tradingSymbol": "INOXWIND30DEC25FUT"
        },
        {
            "pcr": 0.44,
            "tradingSymbol": "IOC30DEC25FUT"
        },
        {
            "pcr": 0.43,
            "tradingSymbol": "IRCTC30DEC25FUT"
        },
        {
            "pcr": 0.4,
            "tradingSymbol": "IREDA30DEC25FUT"
        },
        {
            "pcr": 0.44,
            "tradingSymbol": "I

In [3]:
import json
import pandas as pd
import re

pcr_data = smartApi.putCallRatio()

nifty_fut = []
banknifty_fut = []

if "data" in pcr_data:
    data = pcr_data["data"]
    for item in data:
        tsym = item.get("tradingSymbol", "").upper()
        # Match only "NIFTY" followed by numbers and "FUT" (not FINNIFTY, MIDCPNIFTY, etc.)
        if re.match(r"^NIFTY\d+[A-Z]+\d+FUT$", tsym):
            nifty_fut.append(item)
        elif re.match(r"^BANKNIFTY\d+[A-Z]+\d+FUT$", tsym):
            banknifty_fut.append(item)

# Convert to DataFrames and display
if nifty_fut:
    print("\nNIFTY FUTURE PCR DATA (DataFrame):")
    display(pd.DataFrame(nifty_fut))
if banknifty_fut:
    print("\nBANKNIFTY FUTURE PCR DATA (DataFrame):")
    display(pd.DataFrame(banknifty_fut))


NIFTY FUTURE PCR DATA (DataFrame):


,pcr,tradingSymbol
0,1.28,NIFTY30DEC25FUT



BANKNIFTY FUTURE PCR DATA (DataFrame):


,pcr,tradingSymbol
0,0.97,BANKNIFTY30DEC25FUT


# Trying for integration into dashboard

In [4]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests

class BankNiftyDashboard:
    def __init__(self, api_key, username, pwd, token, file_path, offset_rows=0):
        self.api_key = api_key
        self.username = username
        self.pwd = pwd
        self.token = token
        self.file_path = file_path
        self.offset_rows = offset_rows

        self.symbol_token_map = {
            "HDFCBANK": "1333", "ICICIBANK": "4963", "AXISBANK": "5900", "KOTAKBANK": "1922",
            "SBIN": "3045", "INDUSINDBK": "5258", "BANDHANBNK": "2263", "PNB": "10666",
            "IDFCFIRSTB": "11184", "AUBANK": "21238", "FEDERALBNK": "1023", "BANKBARODA": "4668"
        }
        self.bank_nifty_stocks = list(self.symbol_token_map.keys())
        self.weight_map = {}
        self.last_weightage_fetch = None

        self.smartApi = SmartConnect(self.api_key)
        self.authenticate()
        self.wb, self.sheet = self.initialize_excel()

    def get_worksheet(self):
        import win32com.client
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = True
        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        return ws, wb

    def authenticate(self):
        try:
            totp = pyotp.TOTP(self.token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e

        data = self.smartApi.generateSession(self.username, self.pwd, totp)
        if data['status'] == False:
            logger.error(data)
            raise Exception("Authentication failed")
        else:
            self.authToken = data['data']['jwtToken']
            self.refreshToken = data['data']['refreshToken']
            self.feedToken = self.smartApi.getfeedToken()
            self.profile = self.smartApi.getProfile(self.refreshToken)
            self.smartApi.generateToken(self.refreshToken)

    def get_banknifty_weightage(self):
        if self.last_weightage_fetch == datetime.now().date():
            return

        url = "https://www.smart-investing.in/indices-bse-nse.php?index=BANKNIFTY"
        try:
            tables = pd.read_html(url)
            for table in tables:
                cols = [col.lower() for col in table.columns.astype(str)]
                if "company" in cols and any("weight" in c for c in cols):
                    table.columns = [col.strip() for col in table.columns]
                    table = table.rename(columns={
                        table.columns[0]: "Bank Name",
                        table.columns[1]: "Weightage (%)"
                    })
                    table["Weightage (%)"] = table["Weightage (%)"].str.replace("%", "").astype(float)

                    alias_map = {
                        "HDFC BANK": "HDFCBANK",
                        "ICICI BANK": "ICICIBANK",
                        "AXIS BANK": "AXISBANK",
                        "KOTAK MAHINDRA BANK": "KOTAKBANK",
                        "STATE BANK OF INDIA": "SBIN",
                        "INDUSIND BANK": "INDUSINDBK",
                        "BANDHAN BANK": "BANDHANBNK",
                        "PUNJAB NATIONAL BANK": "PNB",
                        "IDFC FIRST BANK": "IDFCFIRSTB",
                        "AU SMALL FINANCE BANK": "AUBANK",
                        "THE FEDERAL BANK": "FEDERALBNK",
                        "BANK OF BARODA": "BANKBARODA"
                    }

                    self.weight_map.clear()
                    for _, row in table.iterrows():
                        full_name = row["Bank Name"].upper().replace(" LTD", "").strip()
                        symbol = alias_map.get(full_name, None)
                        if symbol:
                            self.weight_map[symbol] = row["Weightage (%)"]
                    self.last_weightage_fetch = datetime.now().date()
                    return
            print("Weightage table not found")
        except Exception as e:
            print("Error fetching weightage:", e)

    def initialize_excel(self):
        excel = win32com.client.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True

        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

        for wb in excel.Workbooks:
            if wb.FullName.lower() == self.file_path.lower():
                return wb, wb.Worksheets(1)
        try:
            wb = excel.Workbooks.Open(self.file_path)
            return wb, wb.Worksheets(1)
        except:
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

    def get_status(self, open, close):
        if (close > open):
            return "Bullish"
        elif (close < open):
            return "Bearish"
        return "Neutral"

    def update_excel(self):
        ws, wb = self.get_worksheet()

        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day

        now = datetime.now()
        today_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)

        if now.weekday() >= 5:
            print("⚠️ Market is closed on Saturday and Sunday.")
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < today_open:
                print("⚠️ Market has not opened yet. Try again after 9:15 AM.")
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                print("⚠️ Market is closed for the day.")
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")

        self.get_banknifty_weightage()
        # ws.UsedRange.Clear()
        headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
        for col, header in enumerate(headers, 1):
            ws.Cells(1 + self.offset_rows, col).Value = header

        row_index = 2 + self.offset_rows
        for stock in self.bank_nifty_stocks:
            try:
                histParam = {
                    "exchange": "NSE",
                    "symboltoken": self.symbol_token_map[stock],
                    "interval": "ONE_MINUTE",
                    "fromdate": from_date,
                    "todate": to_date
                }
                response = self.smartApi.getCandleData(histParam)
                if response['status'] and response['data']:
                    candle = response['data'][-1]
                    ltp = candle[4]
                    volume = candle[5]
                    change_percent = ((candle[4] - candle[1]) / candle[1]) * 100
                    status = self.get_status(candle[1], candle[4]) 
                    weight = self.weight_map.get(stock.upper(), "")
                    ws.Cells(row_index, 1).Value = stock
                    ws.Cells(row_index, 2).Value = ltp
                    ws.Cells(row_index, 3).Value = volume
                    ws.Cells(row_index, 4).Value = round(change_percent, 2)
                    ws.Cells(row_index, 5).Value = status
                    ws.Cells(row_index, 6).Value = weight
                    row_index += 1
                else:
                    print(f"⚠️ Failed to get {stock} ONE_MINUTE data:", response)
            except Exception as e:
                print(f"❌ Error fetching {stock} data:", e)

        ws.Cells(row_index + 1, 1).Value = "Data Fetched At"
        ws.Cells(row_index + 1, 2).Value = to_date

        wb.Save()
        print(f"✅ Bank Nifty data updated in Excel at {datetime.now().strftime('%H:%M:%S')}")

    def run(self, interval_sec=60):
        try:
            while True:
                self.update_excel()
                time.sleep(interval_sec)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

class IndicesDashboard:
    def __init__(self, api_key, username, pwd, token, file_path):
        self.api_key = api_key
        self.username = username
        self.pwd = pwd
        self.token = token
        self.smartApi = SmartConnect(self.api_key)
        self.authenticate()
        self.file_path = file_path
        self.time_intervals = [
            "ONE_MINUTE", "THREE_MINUTE", "FIVE_MINUTE",
            "FIFTEEN_MINUTE", "THIRTY_MINUTE", "ONE_HOUR"
        ]
        latest_future = get_latest_banknifty_future()
        self.instruments = {
            "BANKNIFTY": {"token": "99926009", "exchange": "NSE"},
            "BANKNIFTY FUTURE": {"token": "56782", "exchange": "NFO"},
            "NIFTY": {"token": "99926000", "exchange": "NSE"},
            "INDIA VIX": {"token": "99926017", "exchange": "NSE"}
        }
        self.interval_mapping = {
            "ONE_MINUTE": "1min",
            "THREE_MINUTE": "3min",
            "FIVE_MINUTE": "5min",
            "FIFTEEN_MINUTE": "15min",
            "THIRTY_MINUTE": "30min",
            "ONE_HOUR": "1hr"
        }
        self.setup_excel_structure()

    def get_latest_banknifty_future(self):
        """
        Fetches the latest BANKNIFTY FUTURE token, symbol, expiry, and exchange segment.
        Returns a dict: {"token": ..., "symbol": ..., "expiry": ..., "exch_seg": ...}
        """
        import requests
        import pandas as pd

        url = 'https://margincalculator.angelone.in/OpenAPI_File/files/OpenAPIScripMaster.json'
        data = requests.get(url).json()
        df = pd.DataFrame(data)

        # Filter for BANKNIFTY futures (exclude options)
        banknifty_futures = df[df['symbol'].str.contains(r'^BANKNIFTY.*FUT', na=False)]
        # Get the latest expiry (sort by expiry descending)
        banknifty_futures = banknifty_futures.sort_values('expiry', ascending=False)
        if not banknifty_futures.empty:
            row = banknifty_futures.iloc[0]
            return {
                "token": str(row['token']),
                "symbol": row['symbol'],
                "expiry": row['expiry'],
                "exch_seg": row['exch_seg']
            }
        else:
            return None

    def clear_dashboard(self):
        ws, wb = self.get_worksheet()
        ws.UsedRange.Clear()
        wb.Save()
        print("Dashboard cleared.")

    def authenticate(self):
        try:
            totp = pyotp.TOTP(self.token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e
        data = self.smartApi.generateSession(self.username, self.pwd, totp)
        if data['status'] == False:
            logger.error(data)
            raise Exception("Authentication failed")
        else:
            self.authToken = data['data']['jwtToken']
            self.refreshToken = data['data']['refreshToken']
            self.feedToken = self.smartApi.getfeedToken()
            self.profile = self.smartApi.getProfile(self.refreshToken)
            self.smartApi.generateToken(self.refreshToken)

    def get_worksheet(self):
        import win32com.client
        excel = win32com.client.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        return ws, wb
    
    def initialize_excel(self):
        excel = win32com.client.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        if os.path.exists(self.file_path):
            wb = excel.Workbooks.Open(self.file_path)
            print(f"Opened existing workbook: {self.file_path}")
        else:
            wb = excel.Workbooks.Add()
            wb.SaveAs(self.file_path)
            print(f"Created new workbook: {self.file_path}")
        ws = wb.Sheets(1)
        self.setup_excel_structure()
        return excel, wb, ws

    def setup_excel_structure(self):
        ws, wb = self.get_worksheet()
        ws.Name = "Indices Dashboard"
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        headers = ["", "", "1min", "3min", "5min", "15min", "30min", "1hrs"]
        for col_idx, header in enumerate(headers, 1):
            ws.Cells(4, col_idx).Value = header
        row_labels = {6: "A", 8: "B", 10: "C", 12: "D", 14: "F"}
        indice_names = {6: "BANKNIFTY", 8: "BANKNIFTY FUTURE", 10: "NIFTY", 12: "GIFT NIFTY", 14: "VIX"}
        for col in range(2, 9):
            ws.Columns(col).ColumnWidth = 20
            ws.Columns(col).WrapText = True
        for row, label in row_labels.items():
            ws.Cells(row, 1).Value = label
        for row, label in indice_names.items():
            ws.Cells(row, 2).Value = label
        gradient_pairs = [6, 8, 10, 12, 14]
        base_color = 0xD3D3D3
        def darken_color(hex_color, step):
            r = (hex_color & 0xFF0000) >> 16
            g = (hex_color & 0x00FF00) >> 8
            b = (hex_color & 0x0000FF)
            r = max(0, r - step)
            g = max(0, g - step)
            b = max(0, b - step)
            return (r << 16) + (g << 8) + b
        current_color = base_color
        for top_row in gradient_pairs:
            for col in range(1, 11):
                ws.Cells(top_row, col).Interior.Color = current_color
                ws.Cells(top_row + 1, col).Interior.Color = current_color
            current_color = darken_color(current_color, 15)
        wb.Save()

    def fetch_candle_data(self, symbol, token, exchange, interval):
        now = datetime.now()
        market_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)
        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day
        if now.weekday() >= 5:
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < market_open:
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")
        params = {
            "exchange": exchange,
            "symboltoken": token,
            "interval": interval,
            "fromdate": from_date,
            "todate": to_date
        }
        try:
            response = self.smartApi.getCandleData(params)
            if response['status'] and response['data']:
                data = response['data']
                latest = data[-1:]
                df = pd.DataFrame(latest, columns=["Datetime", "Open", "High", "Low", "Close", "Volume"])
                df["Symbol"] = symbol
                df["Interval"] = interval
                return df
            else:
                print(f"⚠️ No candle data for {symbol} ({interval})")
                return pd.DataFrame()
        except Exception as e:
            print(f"❌ Error fetching {symbol} [{interval}]: {e}")
            return pd.DataFrame()

    def fetch_and_process_data(self):
        final_df = pd.DataFrame()
        for symbol, info in self.instruments.items():
            for interval in self.time_intervals:
                df = self.fetch_candle_data(symbol, info["token"], info["exchange"], interval)
                final_df = pd.concat([final_df, df], ignore_index=True)
        final_df = final_df[["Symbol", "Interval", "Datetime", "Open", "High", "Low", "Close", "Volume"]]
        final_df.sort_values(by=["Symbol", "Interval", "Datetime"], inplace=True)
        return final_df

    def calculate_sentiment(self, df):
        
        def sentiment_logic(row):
            symbol = row["Symbol"]
            close_price = row["Close"]
            open_price = row["Open"]

            if (open_price < close_price):
                sentiment = f"Bullish"
            elif (open_price > close_price):
                sentiment = f"Bearish"
            else:
                sentiment = f"Neutral"

            return sentiment

        df["Sentiment"] = df.apply(sentiment_logic, axis=1)
        df["Interval"] = df["Interval"].map(self.interval_mapping)
        return df

    def update_excel(self, df):
        ws, wb = self.get_worksheet()
        cell_mapping = {
            "BANKNIFTY": {"1min": "C6", "3min": "D6", "5min": "E6", "15min": "F6", "30min": "G6", "1hr": "H6"},
            "BANKNIFTY FUTURE": {"1min": "C8", "3min": "D8", "5min": "E8", "15min": "F8", "30min": "G8", "1hr": "H8"},
            "NIFTY": {"1min": "C10", "3min": "D10", "5min": "E10", "15min": "F10", "30min": "G10", "1hr": "H10"},
            "INDIA VIX": {"1min": "C14", "3min": "D14", "5min": "E14", "15min": "F14", "30min": "G14", "1hr": "H14"}
        }
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        for symbol in cell_mapping.keys():
            for interval, cell in cell_mapping[symbol].items():
                try:
                    sentiment = df[(df["Symbol"] == symbol) & (df["Interval"] == interval)]["Sentiment"].iloc[0]
                except IndexError:
                    print(f"Warning: No sentiment data for {symbol} ({interval})")
                    continue
                ws.Range(cell).Value = sentiment
                if "bullish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x008000  # Green
                elif "bearish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x0000FF  # Blue
                else:
                    ws.Range(cell).Font.Color = 0x000000  # Black
        wb = ws.Parent
        wb.Save()
        print("Excel file updated successfully.")

    def run(self):
        try:
            while True:
                try:
                    df = self.fetch_and_process_data()
                    df = self.calculate_sentiment(df)
                    self.update_excel(self.ws, df)
                    print(f"Data updated at {datetime.now().strftime('%H:%M:%S')}")
                except Exception as e:
                    print(f"Error during data update: {str(e)}")
                    if isinstance(e, requests.exceptions.ConnectionError):
                        print("Connection error detected. Retrying in 60 seconds...")
                time.sleep(60)
        except KeyboardInterrupt:
            print("Script interrupted by user.")
            try:
                self.wb.Save()
                self.wb.Close()
                self.excel.Quit()
                print("Excel application closed successfully.")
            except Exception as e:
                print(f"Error closing Excel: {str(e)}")
        except Exception as e:
            print(f"Fatal error: {str(e)}")
            try:
                self.wb.Close()
                self.excel.Quit()
            except:
                pass

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x9
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 9) - <class 'AttributeError'>: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x9' has no attribute 'CLSIDToClassMap'
Checking 45541000-5750-5300-4B49-4E47534F4655x0x3x0
Could not add module (IID('{45541000-5750-5300-4B49-4E47534F4655}'), 0, 3, 0) - <class 'AttributeError'>: module 'win32com.gen_py.45541000-5750-5300-4B49-4E47534F4655x0x3x0' has no attribute 'CLSIDToClassMap'
Done.


In [5]:
import json
import pandas as pd
import re
import time
from datetime import datetime
import win32com.client as win32

class PCRDashboard:
    def __init__(self, smartApi, file_path, offset_rows=0):
        self.smartApi = smartApi
        self.file_path = file_path
        self.offset_rows = offset_rows

    def fetch_pcr_data(self):
        pcr_data = self.smartApi.putCallRatio()
        nifty_fut = []
        banknifty_fut = []
        if "data" in pcr_data:
            data = pcr_data["data"]
            for item in data:
                tsym = item.get("tradingSymbol", "").upper()
                # Only NIFTY futures (not FINNIFTY, MIDCPNIFTY, etc.)
                if re.match(r"^NIFTY\d+[A-Z]+\d+FUT$", tsym):
                    nifty_fut.append(item)
                elif re.match(r"^BANKNIFTY\d+[A-Z]+\d+FUT$", tsym):
                    banknifty_fut.append(item)
        return pd.DataFrame(nifty_fut), pd.DataFrame(banknifty_fut)

    def write_to_excel(self):
        nifty_df, banknifty_df = self.fetch_pcr_data()
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        if not win32.os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        row = 1 + self.offset_rows
        # Write NIFTY FUTURE PCR
        if not nifty_df.empty:
            ws.Cells(row, 1).Value = "NIFTY FUTURE PCR DATA"
            for col, col_name in enumerate(nifty_df.columns, 1):
                ws.Cells(row + 1, col).Value = col_name
            for i, record in nifty_df.iterrows():
                for j, value in enumerate(record, 1):
                    ws.Cells(row + 2 + i, j).Value = value
            row += 2 + len(nifty_df)
        # Write BANKNIFTY FUTURE PCR
        if not banknifty_df.empty:
            ws.Cells(row, 1).Value = "BANKNIFTY FUTURE PCR DATA"
            for col, col_name in enumerate(banknifty_df.columns, 1):
                ws.Cells(row + 1, col).Value = col_name
            for i, record in banknifty_df.iterrows():
                for j, value in enumerate(record, 1):
                    ws.Cells(row + 2 + i, j).Value = value
        wb.Save()
        print(f"PCR data written to Excel at offset {self.offset_rows}")

In [6]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}

class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30,
        pcr_offset=50
    ):
        self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        self.indices_dashboard = IndicesDashboard(**indices_args)
        self.indices_dashboard.clear_dashboard()
        self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        # PCR Dashboard
        self.pcr_dashboard = PCRDashboard(
            self.banknifty_dashboard.smartApi,
            banknifty_args["file_path"], 
            offset_rows=pcr_offset
        )
        #self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path)
        #self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):
        try:
            while True:
                try:
                    print("Updating Indices Dashboard...")
                    df = self.indices_dashboard.fetch_and_process_data()
                    df = self.indices_dashboard.calculate_sentiment(df)
                    self.indices_dashboard.update_excel(df)
                    print("Updating BankNifty Dashboard...")
                    self.banknifty_dashboard.update_excel()
                    print("Updating PCR Dashboard...")
                    self.pcr_dashboard.write_to_excel()
                    print(f"All dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    # self.bankniftyupdater.run()
                    # self.niftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=30, pcr_offset=50)

[I 251201 17:40:59 smartConnect:124] in pool


AttributeError: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x9' has no attribute 'MinorVersion'

In [7]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
from SmartApi import SmartConnect
from logzero import logger
import re

class BankNiftyDashboard:
    def __init__(self, smartApi, file_path, offset_rows=0):
        self.smartApi = smartApi
        self.file_path = file_path
        self.offset_rows = offset_rows

        self.symbol_token_map = {
            "HDFCBANK": "1333", "ICICIBANK": "4963", "AXISBANK": "5900", "KOTAKBANK": "1922",
            "SBIN": "3045", "INDUSINDBK": "5258", "BANDHANBNK": "2263", "PNB": "10666",
            "IDFCFIRSTB": "11184", "AUBANK": "21238", "FEDERALBNK": "1023", "BANKBARODA": "4668"
        }
        self.bank_nifty_stocks = list(self.symbol_token_map.keys())
        self.weight_map = {}
        self.last_weightage_fetch = None
        self.wb, self.sheet = self.initialize_excel()

    def get_worksheet(self):
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True
        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        return ws, wb

    def get_banknifty_weightage(self):
        if self.last_weightage_fetch == datetime.now().date():
            return

        url = "https://www.smart-investing.in/indices-bse-nse.php?index=BANKNIFTY"
        try:
            tables = pd.read_html(url)
            for table in tables:
                cols = [col.lower() for col in table.columns.astype(str)]
                if "company" in cols and any("weight" in c for c in cols):
                    table.columns = [col.strip() for col in table.columns]
                    table = table.rename(columns={
                        table.columns[0]: "Bank Name",
                        table.columns[1]: "Weightage (%)"
                    })
                    table["Weightage (%)"] = table["Weightage (%)"].str.replace("%", "").astype(float)

                    alias_map = {
                        "HDFC BANK": "HDFCBANK",
                        "ICICI BANK": "ICICIBANK",
                        "AXIS BANK": "AXISBANK",
                        "KOTAK MAHINDRA BANK": "KOTAKBANK",
                        "STATE BANK OF INDIA": "SBIN",
                        "INDUSIND BANK": "INDUSINDBK",
                        "BANDHAN BANK": "BANDHANBNK",
                        "PUNJAB NATIONAL BANK": "PNB",
                        "IDFC FIRST BANK": "IDFCFIRSTB",
                        "AU SMALL FINANCE BANK": "AUBANK",
                        "THE FEDERAL BANK": "FEDERALBNK",
                        "BANK OF BARODA": "BANKBARODA"
                    }

                    self.weight_map.clear()
                    for _, row in table.iterrows():
                        full_name = row["Bank Name"].upper().replace(" LTD", "").strip()
                        symbol = alias_map.get(full_name, None)
                        if symbol:
                            self.weight_map[symbol] = row["Weightage (%)"]
                    self.last_weightage_fetch = datetime.now().date()
                    return
            print("Weightage table not found")
        except Exception as e:
            print("Error fetching weightage:", e)

    def initialize_excel(self):
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True

        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

        for wb in excel.Workbooks:
            if wb.FullName.lower() == self.file_path.lower():
                return wb, wb.Worksheets(1)
        try:
            wb = excel.Workbooks.Open(self.file_path)
            return wb, wb.Worksheets(1)
        except:
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

    def get_status(self, open, close):
        if (close > open):
            return "Bullish"
        elif (close < open):
            return "Bearish"
        return "Neutral"

    def update_excel(self):
        ws, wb = self.get_worksheet()

        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day

        now = datetime.now()
        today_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)

        if now.weekday() >= 5:
            print("⚠️ Market is closed on Saturday and Sunday.")
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < today_open:
                print("⚠️ Market has not opened yet. Try again after 9:15 AM.")
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                print("⚠️ Market is closed for the day.")
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")

        self.get_banknifty_weightage()
        headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
        for col, header in enumerate(headers, 1):
            ws.Cells(1 + self.offset_rows, col).Value = header

        row_index = 2 + self.offset_rows
        for stock in self.bank_nifty_stocks:
            try:
                histParam = {
                    "exchange": "NSE",
                    "symboltoken": self.symbol_token_map[stock],
                    "interval": "ONE_MINUTE",
                    "fromdate": from_date,
                    "todate": to_date
                }
                response = self.smartApi.getCandleData(histParam)
                if response['status'] and response['data']:
                    candle = response['data'][-1]
                    ltp = candle[4]
                    volume = candle[5]
                    change_percent = ((candle[4] - candle[1]) / candle[1]) * 100
                    status = self.get_status(candle[1], candle[4]) 
                    weight = self.weight_map.get(stock.upper(), "")
                    ws.Cells(row_index, 1).Value = stock
                    ws.Cells(row_index, 2).Value = ltp
                    ws.Cells(row_index, 3).Value = volume
                    ws.Cells(row_index, 4).Value = round(change_percent, 2)
                    ws.Cells(row_index, 5).Value = status
                    ws.Cells(row_index, 6).Value = weight
                    row_index += 1
                else:
                    print(f"⚠️ Failed to get {stock} ONE_MINUTE data:", response)
            except Exception as e:
                print(f"❌ Error fetching {stock} data:", e)

        ws.Cells(row_index + 1, 1).Value = "Data Fetched At"
        ws.Cells(row_index + 1, 2).Value = to_date

        wb.Save()
        print(f"✅ Bank Nifty data updated in Excel at {datetime.now().strftime('%H:%M:%S')}")

class IndicesDashboard:
    def __init__(self, smartApi, file_path):
        self.smartApi = smartApi
        self.file_path = file_path
        self.time_intervals = [
            "ONE_MINUTE", "THREE_MINUTE", "FIVE_MINUTE",
            "FIFTEEN_MINUTE", "THIRTY_MINUTE", "ONE_HOUR"
        ]
        self.instruments = {
            "BANKNIFTY": {"token": "99926009", "exchange": "NSE"},
            "BANKNIFTY FUTURE": {"token": "56782", "exchange": "NFO"},
            "NIFTY": {"token": "99926000", "exchange": "NSE"},
            "INDIA VIX": {"token": "99926017", "exchange": "NSE"}
        }
        self.interval_mapping = {
            "ONE_MINUTE": "1min",
            "THREE_MINUTE": "3min",
            "FIVE_MINUTE": "5min",
            "FIFTEEN_MINUTE": "15min",
            "THIRTY_MINUTE": "30min",
            "ONE_HOUR": "1hr"
        }
        self.setup_excel_structure()

    def get_worksheet(self):
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        return ws, wb

    def setup_excel_structure(self):
        ws, wb = self.get_worksheet()
        ws.Name = "Indices Dashboard"
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        headers = ["", "", "1min", "3min", "5min", "15min", "30min", "1hrs"]
        for col_idx, header in enumerate(headers, 1):
            ws.Cells(4, col_idx).Value = header
        row_labels = {6: "A", 8: "B", 10: "C", 12: "D", 14: "F"}
        indice_names = {6: "BANKNIFTY", 8: "BANKNIFTY FUTURE", 10: "NIFTY", 12: "GIFT NIFTY", 14: "VIX"}
        for col in range(2, 9):
            ws.Columns(col).ColumnWidth = 20
            ws.Columns(col).WrapText = True
        for row, label in row_labels.items():
            ws.Cells(row, 1).Value = label
        for row, label in indice_names.items():
            ws.Cells(row, 2).Value = label
        gradient_pairs = [6, 8, 10, 12, 14]
        base_color = 0xD3D3D3
        def darken_color(hex_color, step):
            r = (hex_color & 0xFF0000) >> 16
            g = (hex_color & 0x00FF00) >> 8
            b = (hex_color & 0x0000FF)
            r = max(0, r - step)
            g = max(0, g - step)
            b = max(0, b - step)
            return (r << 16) + (g << 8) + b
        current_color = base_color
        for top_row in gradient_pairs:
            for col in range(1, 11):
                ws.Cells(top_row, col).Interior.Color = current_color
                ws.Cells(top_row + 1, col).Interior.Color = current_color
            current_color = darken_color(current_color, 15)
        wb.Save()

    def fetch_candle_data(self, symbol, token, exchange, interval):
        now = datetime.now()
        market_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)
        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day
        if now.weekday() >= 5:
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < market_open:
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")
        params = {
            "exchange": exchange,
            "symboltoken": token,
            "interval": interval,
            "fromdate": from_date,
            "todate": to_date
        }
        try:
            response = self.smartApi.getCandleData(params)
            if response['status'] and response['data']:
                data = response['data']
                latest = data[-1:]
                df = pd.DataFrame(latest, columns=["Datetime", "Open", "High", "Low", "Close", "Volume"])
                df["Symbol"] = symbol
                df["Interval"] = interval
                return df
            else:
                print(f"⚠️ No candle data for {symbol} ({interval})")
                return pd.DataFrame()
        except Exception as e:
            print(f"❌ Error fetching {symbol} [{interval}]: {e}")
            return pd.DataFrame()

    def fetch_and_process_data(self):
        final_df = pd.DataFrame()
        for symbol, info in self.instruments.items():
            for interval in self.time_intervals:
                df = self.fetch_candle_data(symbol, info["token"], info["exchange"], interval)
                final_df = pd.concat([final_df, df], ignore_index=True)
        final_df = final_df[["Symbol", "Interval", "Datetime", "Open", "High", "Low", "Close", "Volume"]]
        final_df.sort_values(by=["Symbol", "Interval", "Datetime"], inplace=True)
        return final_df

    def calculate_sentiment(self, df):
        def sentiment_logic(row):
            symbol = row["Symbol"]
            close_price = row["Close"]
            open_price = row["Open"]
            if (open_price < close_price):
                sentiment = f"Bullish"
            elif (open_price > close_price):
                sentiment = f"Bearish"
            else:
                sentiment = f"Neutral"
            return sentiment

        df["Sentiment"] = df.apply(sentiment_logic, axis=1)
        df["Interval"] = df["Interval"].map(self.interval_mapping)
        return df

    def update_excel(self, df):
        ws, wb = self.get_worksheet()
        cell_mapping = {
            "BANKNIFTY": {"1min": "C6", "3min": "D6", "5min": "E6", "15min": "F6", "30min": "G6", "1hr": "H6"},
            "BANKNIFTY FUTURE": {"1min": "C8", "3min": "D8", "5min": "E8", "15min": "F8", "30min": "G8", "1hr": "H8"},
            "NIFTY": {"1min": "C10", "3min": "D10", "5min": "E10", "15min": "F10", "30min": "G10", "1hr": "H10"},
            "INDIA VIX": {"1min": "C14", "3min": "D14", "5min": "E14", "15min": "F14", "30min": "G14", "1hr": "H14"}
        }
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        for symbol in cell_mapping.keys():
            for interval, cell in cell_mapping[symbol].items():
                try:
                    sentiment = df[(df["Symbol"] == symbol) & (df["Interval"] == interval)]["Sentiment"].iloc[0]
                except IndexError:
                    print(f"Warning: No sentiment data for {symbol} ({interval})")
                    continue
                ws.Range(cell).Value = sentiment
                if "bullish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x008000  # Green
                elif "bearish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x0000FF  # Blue
                else:
                    ws.Range(cell).Font.Color = 0x000000  # Black
        wb = ws.Parent
        wb.Save()
        print("Excel file updated successfully.")

class PCRDashboard:
    def __init__(self, smartApi, file_path, offset_rows=0):
        self.smartApi = smartApi
        self.file_path = file_path
        self.offset_rows = offset_rows

    def fetch_pcr_data(self):
        pcr_data = self.smartApi.putCallRatio()
        nifty_fut = []
        banknifty_fut = []
        if "data" in pcr_data:
            data = pcr_data["data"]
            for item in data:
                tsym = item.get("tradingSymbol", "").upper()
                # Only NIFTY futures (not FINNIFTY, MIDCPNIFTY, etc.)
                if re.match(r"^NIFTY\d+[A-Z]+\d+FUT$", tsym):
                    nifty_fut.append(item)
                elif re.match(r"^BANKNIFTY\d+[A-Z]+\d+FUT$", tsym):
                    banknifty_fut.append(item)
        return pd.DataFrame(nifty_fut), pd.DataFrame(banknifty_fut)

    def write_to_excel(self):
        nifty_df, banknifty_df = self.fetch_pcr_data()
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            ws = wb.Worksheets(1)
            wb.SaveAs(self.file_path)
        else:
            wb = excel.Workbooks.Open(self.file_path)
            ws = wb.Worksheets(1)
        row = 1 + self.offset_rows
        # Write NIFTY FUTURE PCR
        if not nifty_df.empty:
            ws.Cells(row, 1).Value = "NIFTY FUTURE PCR DATA"
            for col, col_name in enumerate(nifty_df.columns, 1):
                ws.Cells(row + 1, col).Value = col_name
            for i, record in nifty_df.iterrows():
                for j, value in enumerate(record, 1):
                    ws.Cells(row + 2 + i, j).Value = value
            row += 2 + len(nifty_df)
        # Write BANKNIFTY FUTURE PCR
        if not banknifty_df.empty:
            ws.Cells(row, 1).Value = "BANKNIFTY FUTURE PCR DATA"
            for col, col_name in enumerate(banknifty_df.columns, 1):
                ws.Cells(row + 1, col).Value = col_name
            for i, record in banknifty_df.iterrows():
                for j, value in enumerate(record, 1):
                    ws.Cells(row + 2 + i, j).Value = value
        wb.Save()
        print(f"PCR data written to Excel at offset {self.offset_rows}")

class DashboardIntegrator:
    def __init__(
        self,
        api_key,
        username,
        pwd,
        token,
        file_path,
        banknifty_offset=15,
        pcr_offset=50,
        banknifty_interval=30
    ):
        self.smartApi = SmartConnect(api_key)
        self.authenticate(username, pwd, token)
        self.banknifty_dashboard = BankNiftyDashboard(self.smartApi, file_path, offset_rows=banknifty_offset)
        self.indices_dashboard = IndicesDashboard(self.smartApi, file_path)
        self.pcr_dashboard = PCRDashboard(self.smartApi, file_path, offset_rows=pcr_offset)
        self.banknifty_interval = banknifty_interval
        self.indices_dashboard.setup_excel_structure()

    def authenticate(self, username, pwd, token):
        try:
            totp = pyotp.TOTP(token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e
        data = self.smartApi.generateSession(username, pwd, totp)
        if data['status'] == False:
            logger.error(data)
            raise Exception("Authentication failed")
        else:
            self.authToken = data['data']['jwtToken']
            self.refreshToken = data['data']['refreshToken']
            self.feedToken = self.smartApi.getfeedToken()
            self.profile = self.smartApi.getProfile(self.refreshToken)
            self.smartApi.generateToken(self.refreshToken)

    def run(self):
        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    print("Updating PCR Dashboard...")
                    self.pcr_dashboard.write_to_excel()
                    print(f"All dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

# Usage:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx"
integrator = DashboardIntegrator(
    api_key='3tA3noq3',
    username='AAAF620196',
    pwd='0054',
    token='YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    file_path=file_path,
    banknifty_offset=15,
    pcr_offset=32,
    banknifty_interval=30
)
integrator.run()

[I 251201 17:41:14 smartConnect:124] in pool


AttributeError: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x9' has no attribute 'MinorVersion'